In [1]:
import pulp
from pulp import LpStatus, LpStatusOptimal
import pickle
import os
_solution_id = 0
TMP_FOLDER = 'tmp'

ZERO_TOLERANCE = 1e-10


def generate_upper_bound_for_efficiency_score():
    ''' Returns upper bound for efficiency score for usual envelopment model.

        Note:
            This function only works with input-oriented envelopment models.

        Returns:
            double: 1, since efficiency score must be <= 1 for usual
                input-oriented envelopment model.
    '''
    return 1


def generate_supper_efficiency_upper_bound():
    ''' Returns upper bound for efficiency score for a super efficiency
        model.

        Note:
            This function only works with input-oriented envelopment models.

        Returns:
            None, since efficiency score might be greater than 1 for
            super efficiency input-oriented envelopment model.
    '''
    return None


def generate_lower_bound_for_efficiency_score():
    ''' Returns lower bound of the inverse of efficiency score for
        usual envelopment model.

        Note:
            This function only works with output-oriented envelopment models.

        Returns:
            double: 1, since inverse of efficiency score must be > 1 for
                usual output-oriented envelopment model.
    '''
    return 1


def generate_supper_efficiency_lower_bound():
    ''' Returns lower bound of the inverse of efficiency score for
        a super efficiency envelopment model.

        Note:
            This function only works with output-oriented envelopment models.

        Returns:
            double: 0, since efficiency score might be greater
                than 1 for super efficient output-oriented envelopment model.
    '''
    return 0

def is_efficient(efficiency_score, lambda_variable):
    ''' Checks if dmu with given efficiency score and value of
        lambda variable is efficient.

        Args:
            efficiency_score (double): efficiency spyDEA.core.
            lambda_variable (double): value of lambda variable corresponding
                to DMU under consideration.

        Returns:
            bool: True if DMU is efficient, False otherwise.

        Example:
            >>> is_efficient(1, 1)
            True
            >>> is_efficient(0.5, 0)
            False
            >>> is_efficient(0.9999999, 1)
            True
            >>> is_efficient(1.0000001, 1)
            True

    '''
    if efficiency_score == 1:
        return True
    if lambda_variable > ZERO_TOLERANCE:
        return True
    # it seems that lambda_variable is not ultimate indication of efficiency
    if efficiency_score > 1:
        return True
    return False


class Solution(object):
    ''' This class implements basic solution.

        Attributes:
            _solution_id (int): solution ID.
            orientation (str): problem orientation, can take values
                input or output.
            _input_data (InputData): object that stores input data.
            efficiency_scores (dict of str to double): dictionary that maps
                DMU code to efficiency spyDEA.core.
            lp_status (dict of str to pulp.LpStatus): dictionary that maps
                DMU code to LP status (optimal, unbounded, etc).
            input_duals (dict of str to dict of str to double): dictionary
                that maps DMU code to another dictionary that maps input
                category name to value of dual variable.
            output_duals (dict of str to dict of str to double): dictionary
                that maps DMU code to another dictionary that maps output
                category name to value of dual variable.
            return_to_scale (dict of str to str): dictionary that maps DMU code
                to the return-to-scale of the DMU

        Args:
            input_data (InputData): object that stores input data.
    '''
    def __init__(self, input_data):

        global _solution_id
        _solution_id += 1
        self._solution_id = _solution_id
        self.orientation = ''
        self._input_data = input_data

        self.efficiency_scores = dict()
        self.lp_status = dict()
        self.input_duals = dict()
        self.output_duals = dict()
        self.return_to_scale = dict()
        for dmu_code in input_data.DMU_codes:
            self.input_duals[dmu_code] = dict()
            self.output_duals[dmu_code] = dict()

        if not os.path.exists(TMP_FOLDER):
            os.makedirs(TMP_FOLDER)

    def add_efficiency_score(self, dmu_code, efficiency_score):
        ''' Adds efficiency score of a given DMU to internal
            data structure.

            Args:
                dmu_code (str): DMU code.
                efficiency_score (double): efficiency spyDEA.core.

            Raises:
                ValueError: if dmu_code does not exist or has invalid
                    value.
        '''
        self._check_efficiency_score(efficiency_score)
        self._check_if_dmu_code_exists(dmu_code)
        self.efficiency_scores[dmu_code] = efficiency_score

    def _check_efficiency_score(self, efficiency_score):
        ''' Checks if efficiency score has a valid value.

            Args:
                efficiency_score (double): efficiency spyDEA.core.

            Raises:
                ValueError: if efficiency score has invalid value.
        '''
        if efficiency_score < 0 or efficiency_score > 1:
            raise ValueError('Efficiency score must be within [0, 1]')

    def get_efficiency_score(self, dmu_code):
        ''' Returns efficiency score of a given DMU.

            Args:
                dmu_code (str): DMU code.

            Returns:
                double: efficiency spyDEA.core.
        '''
        return self.efficiency_scores[dmu_code]

    def is_efficient(self, dmu_code, lambda_variables=None):
        ''' Checks if a given DMU is efficient.

            Args:
                dmu_code (str): DMU code.
                lambda_variables (dict of str to double, optional): dictionary
                    that maps DMU codes to the corresponding value
                    of lambda variables. If it is not given, it will be loaded
                    from a pickled file.

            Returns:
                bool: True if a given DMU is efficient, False otherwise.
        '''
        if self.lp_status[dmu_code] != LpStatusOptimal:
            return False
        file_name = self._get_pickle_name(dmu_code)
        if not lambda_variables:
            lambda_variables = pickle.load(open(file_name, 'rb'))
        return is_efficient(self.get_efficiency_score(dmu_code),
                            lambda_variables.get(dmu_code, 0))

    def add_lambda_variables(self, dmu_code, variables):
        ''' Adds lambda variables corresponding to a given DMU
            to pickled file.

            Args:
                dmu_code (str): DMU code.
                variables (dict of str to double): dictionary
                    that maps DMU codes to the corresponding value
                    of lambda variables.

            Raises:
                ValueError: if DMU code does not exist or
                    if number of lambda variables is not equal
                    to total number of DMU codes, or
                    if variables contain keys that are not existing DMU codes.
        '''
        self._check_if_dmu_code_exists(dmu_code)
        self._validate_lambda_variables(variables)
        with open(self._get_pickle_name(dmu_code), 'wb') as f:
            pickle.dump(variables, f)

    def get_lambda_variables(self, dmu_code):
        ''' Returns lambda variables corresponding to a given DMU.

            Args:
                dmu_code (str): DMU code.

            Returns:
                dict of str to double: lambda variables.
        '''
        file_name = self._get_pickle_name(dmu_code)
        return pickle.load(open(file_name, 'rb'))

    def _get_pickle_name(self, dmu_code):
        ''' Generates a unique name for pickled file with lambda
            variables.

            Args:
                dmu_code (str): DMU code.

            Returns:
                str: generated file name.
        '''
        file_name = 'lambda{0}_{1}.p'.format(self._solution_id, dmu_code)
        return os.path.join(TMP_FOLDER, file_name)

    def _check_if_dmu_code_exists(self, dmu_code):
        ''' Checks if a given DMU code exists.

            Args:
                dmu_code (str): DMU code.

            Raises:
                ValueError: if a given DMU code does not exist.
        '''
        if dmu_code not in self._input_data.DMU_codes:
            raise ValueError('DMU code {dmu} does not exist'.format(
                dmu=dmu_code))

    def _validate_lambda_variables(self, variables):
        ''' Checks if variables contain existing DMU codes as keys.

            Args:
                variables (dict of str to double): dictionary
                    that maps DMU codes to the corresponding value
                    of lambda variables.

            Raises:
                ValueError: if variables contain non-existing DMU codes.
        '''
        for key in variables.keys():
            self._check_if_dmu_code_exists(key)

    def add_input_dual(self, dmu_code, input_category, dual_value):
        ''' Adds value of a dual variable associated with a given input category
            and DMU code to internal data structure.

            Args:
                dmu_code (str): DMU code.
                input_category (str): input category name.
                dual_value (double): value of a dual variable.

            Raises:
                ValueError: if a given category is not a valid input category.
        '''
        self._check_if_dmu_code_exists(dmu_code)
        if input_category not in self._input_data.input_categories:
            raise ValueError('{category} is not a valid input category'.format(
                category=input_category))
        self.input_duals[dmu_code][input_category] = dual_value

    def add_output_dual(self, dmu_code, output_category, dual_value):
        ''' Adds value of a dual variable associated with a given output
            category and DMU code to internal data structure.

            Args:
                dmu_code (str): DMU code.
                output_category (str): output category name.
                dual_value (double): value of a dual variable.

            Raises:
                ValueError: if a given category is not a valid output category.
        '''
        self._check_if_dmu_code_exists(dmu_code)
        if output_category not in self._input_data.output_categories:
            raise ValueError('{category} is not a valid output category'.format(
                             category=output_category))
        self.output_duals[dmu_code][output_category] = dual_value

    def get_input_dual(self, dmu_code, input_category):
        ''' Returns dual variable value corresponding to a given DMU and
            input category.

            Args:
                dmu_code (str): DMU code.
                input_category (str): input category name.

            Returns:
                double: dual variable value.
        '''
        return self.input_duals[dmu_code][input_category]

    def get_output_dual(self, dmu_code, output_category):
        ''' Returns dual variable value corresponding to a given DMU and
            output category.

            Args:
                dmu_code (str): DMU code.
                output_category (str): output category name.

            Returns:
                double: dual variable value.
        '''
        return self.output_duals[dmu_code][output_category]

    def add_lp_status(self, dmu_code, lp_status):
        ''' Adds LP status corresponding to a given DMU to internal
            data structure.

            Args:
                dmu_code (str): DMU code.
                lp_status (pulp.LpStatus): LP status.
        '''
        self._check_if_dmu_code_exists(dmu_code)
        self.lp_status[dmu_code] = lp_status

    def _print_for_one_dmu(self, dmu_code):
        ''' Prints on screen all information available for a given DMU.

            Args:
               dmu_code (str): DMU code.
        '''
        print('DMU: {dmu}'.format(
            dmu=self._input_data.get_dmu_user_name(dmu_code)))
        print('code: ', dmu_code)
        if self.lp_status.get(dmu_code):
            print('LP status: {status}'.format(
                status=LpStatus[self.lp_status.get(dmu_code)]))
            if self.lp_status.get(dmu_code) == LpStatusOptimal:
                print('Efficiency score: {score}'.format(
                    score=self.efficiency_scores.get(dmu_code)))
                print('Lambda variables: {vars}'.format(
                    vars=self.get_lambda_variables(dmu_code)))
                print('Input duals: {duals}'.format(
                    duals=self.input_duals.get(dmu_code)))
                print('Output duals: {duals}'.format(
                    duals=self.output_duals.get(dmu_code)))

    def print_solution(self):
        ''' Prints all data on the screen.
        '''
        for dmu_code in self._input_data.DMU_codes:
            self._print_for_one_dmu(dmu_code)

def check_input_and_output_categories(input_data):
    ''' Raises ValueError if input or output categories are empty.

        Args:
            input_data (InputData): objects that stores all input data.

        Raises:
            ValueError: if input or output categories are empty.
    '''
    if (len(input_data.input_categories) == 0 or
            len(input_data.output_categories) == 0):
        raise ValueError('Both input and output categories must be specified')


def do_nothing():
    ''' Helper function. Does nothing.
    '''
    pass


class ModelBase(object):
    ''' Abstract base class for some of the DEA models.

        Attributes:
            input_data (InputData): object that stores all input data.
            update_dmu_str_var (func): function that updates
                solution progress.
            lp_model (pulp.LpProblem): pulp LP.

        Args:
            input_data (InputData): object that stores all input data.
            update_str (func, optional): function that updates
                solution progress. Defaults to a function that does nothing.
    '''
    def __init__(self, input_data, update_str=do_nothing):
        self.input_data = input_data
        self.update_dmu_str_var = update_str
        self.lp_model = None

    def run(self):
        ''' Solves a given problem.
        '''
        check_input_and_output_categories(self.input_data)
        model_solution = self._create_solution()
        self._create_lp()
        for count, dmu_code in enumerate(self.input_data.DMU_codes):
            self.run_for_one_DMU(dmu_code, model_solution)
            # self.lp_model.writeLP("dmu_{0}.txt".format(dmu_code))
            self.update_dmu_str_var()
        return model_solution

    def _create_solution(self):
        ''' Allocates solution object.

            Returns:
                (Solution): created solution object.
        '''
        return Solution(self.input_data)

    def run_for_one_DMU(self, dmu_code, model_solution):
        ''' Solves LP for a given DMU and stores solution.

            Args:
                dmu_code (str): DMU code.
                model_solution (Solution): solution.
        '''
        self._update_lp(dmu_code)
        self.lp_model.solve()
        self._fill_solution(dmu_code, model_solution)

    def _fill_solution(self, dmu_code, model_solution):
        ''' Fills given solution with data calculated for one DMU.
            Must be implemented in derived classes.

            Args:
                dmu_code (str): DMU code for which the LP was solved.
                model_solution (Solution): object where solution for one DMU
                    will be written.
        '''
        raise NotImplementedError()

    def _create_lp(self):
        ''' Creates initial linear program. Must be implemented in derived
            classes.
        '''
        raise NotImplementedError()

    def _update_lp(self, dmu_code):
        ''' Updates existing linear program with coefficients corresponding
            to a given DMU. Must be implemented in derived classes.

            Args:
                dmu_code (str): DMU code.
        '''
        raise NotImplementedError()


class EnvelopmentModelBase(ModelBase):
    ''' This class is a base class for different envelopment models.
        It implements general structure of all envelopment models.
        Concrete variations of envelopment models are passed in the
        class constructor.

        Attributes:
            _concrete_model: concrete implementation of the envelopment
                model.
            _constraint_creator: object that creates a proper
                constraint depending on presence of
                disposable variables.
            _variables (dict of str to pulp.LpVariable): dictionary of pulp
                variables than maps variable names to pulp variables.
            _constraints (dict of str to str): dictionary that maps name of
                the category to the name of the corresponding constraint.
            _should_add_efficiency (bool): if set to False, previously
                stored efficiency score is used. This variable is changed
                in two phase model.

        Args:
            input_data (InputData): object that stores all data of
                a DEA instance.
            concrete_model: concrete implementation of the envelopment
                model.
            constraint_creator: object that creates a proper
                constraint depending on presence of disposable variables.
    '''
    def __init__(self, input_data, concrete_model, constraint_creator):
        super(EnvelopmentModelBase, self).__init__(input_data)
        self._concrete_model = concrete_model
        self._constraint_creator = constraint_creator
        self._variables = dict()
        self._constraints = dict()
        self._should_add_efficiency = True

    def _create_lp(self):
        ''' Creates initial linear program.
        '''
        assert len(self.input_data.DMU_codes) != 0
        self._variables.clear()
        self._constraints.clear()
        # create LP for the first DMU - it is the easiest way to
        # adapt current code
        for elem in self.input_data.DMU_codes:
            dmu_code = elem
            break

        orientation = self._concrete_model.get_orientation()
        obj_type = self._concrete_model.get_objective_type()
        self.lp_model = pulp.LpProblem('Envelopment model: {0}-'
                                       'oriented'.format(orientation),
                                       obj_type)
        ub_obj = self._concrete_model.get_upper_bound_for_objective_variable()
        obj_variable = pulp.LpVariable('Variable in objective function',
                                       self._concrete_model.
                                       get_lower_bound_for_objective_variable(),
                                       ub_obj,
                                       pulp.LpContinuous)
        self._variables['obj_var'] = obj_variable

        self.lp_model += (obj_variable,
                          'Efficiency score or inverse of efficiency score')

        lambda_variables = pulp.LpVariable.dicts('lambda',
                                                 self.input_data.DMU_codes,
                                                 0, None, pulp.LpContinuous)
        self._variables.update(lambda_variables)
        self._add_constraints_for_outputs(lambda_variables, dmu_code,
                                          obj_variable)
        self._add_constraints_for_inputs(lambda_variables, dmu_code,
                                         obj_variable)

    def _update_lp(self, dmu_code):
        ''' Updates existing linear program with coefficients corresponding
            to a given DMU.

            Args:
                dmu_code (str): DMU code.
        '''
        for output_category in self.input_data.output_categories:
            constraint_name = self._constraints[output_category]
            current_output = self.input_data.coefficients[(dmu_code,
                                                          output_category)]
            constraint = self.lp_model.constraints[constraint_name]
            self._concrete_model.update_output_category_coefficient(
                current_output, constraint, self._variables['obj_var'],
                output_category)
        for input_category in self.input_data.input_categories:
            current_input = self.input_data.coefficients[(dmu_code,
                                                         input_category)]
            constraint_name = self._constraints[input_category]
            constraint = self.lp_model.constraints[constraint_name]
            self._concrete_model.update_input_category_coefficient(
                current_input, constraint, self._variables['obj_var'],
                input_category)

    def _add_constraints_for_outputs(self, variables, dmu_code,
                                     obj_variable):
        ''' Adds constraints for outputs to linear program.

            Args:
                variables (dict of str to pulp.LpVariable): a dictionary
                    that maps DMU codes to pulp.LpVariable, created with
                    pulp.LpVariable.dicts.
                dmu_code (str): DMU code for which LP is being created.
                obj_variable (pulp.LpVariable): LP variable that is optimised
                    (either efficiency score or inverse of efficiency score).
        '''
        for (count, output_category) in enumerate(
                self.input_data.output_categories):
            current_output = self.input_data.coefficients[(dmu_code,
                                                          output_category)]
            output_coeff = self._concrete_model.get_output_variable_coefficient(
                obj_variable, output_category)
            sum_all_outputs = pulp.lpSum([variables[dmu] *
                                         self.input_data.coefficients
                                         [(dmu, output_category)]
                                         for dmu in self.input_data.DMU_codes])
            name = 'constraint_output_{count}'.format(count=count)
            self.lp_model += (self._constraint_creator.create(
                              -output_coeff * current_output +
                              sum_all_outputs, 0, output_category), name)
            self._constraints[output_category] = name

    def _add_constraints_for_inputs(self, variables,
                                    dmu_code, obj_variable):
        ''' Adds constraints for inputs to LP.

            Args:
                variables (dict of {str: pulp.LpVariable}): a dictionary that
                    maps DMU codes to pulp.LpVariable, created with
                    pulp.LpVariable.dicts.
                dmu_code (str): DMU code for which LP is being created.
                obj_variable (pulp.LpVariable): LP variable that is optimised
                    (either efficiency score or inverse of efficiency score).
        '''
        for (count, input_category) in enumerate(
                self.input_data.input_categories):
            current_input = self.input_data.coefficients[(dmu_code,
                                                         input_category)]
            input_coeff = self._concrete_model.get_input_variable_coefficient(
                obj_variable, input_category)
            sum_all_inputs = pulp.lpSum([variables[dmu] *
                                        self.input_data.coefficients
                                        [(dmu, input_category)]
                                        for dmu in
                                        self.input_data.DMU_codes])
            name = 'constraint_input_{count}'.format(count=count)
            self.lp_model += (self._constraint_creator.create(
                              input_coeff * current_input
                              - sum_all_inputs, 0, input_category), name)
            self._constraints[input_category] = name

    def _fill_solution(self, dmu_code, model_solution):
        ''' Fills given solution with data calculated for one DMU.

            Args:
                dmu_code (str): DMU code for which the LP was solved.
                model_solution (Solution): object where solution for one DMU
                    will be written.
        '''
        model_solution.orientation = self._concrete_model.get_orientation()
        model_solution.add_lp_status(dmu_code, self.lp_model.status)

        if self.lp_model.status == pulp.LpStatusOptimal:
            lambda_variables = dict()
            for dmu in self.input_data.DMU_codes:
                var = self._variables.get(dmu, None)
                if (var is not None and var.varValue is not None and
                        abs(var.varValue) > ZERO_TOLERANCE):
                    lambda_variables[dmu] = var.varValue

            if self._should_add_efficiency:
                model_solution.add_efficiency_score(
                    dmu_code, self._concrete_model.process_obj_var
                    (pulp.value(self.lp_model.objective)))
                
            model_solution.add_lambda_variables(dmu_code, lambda_variables)
            self._process_duals(dmu_code, self.input_data.input_categories,
                                model_solution.add_input_dual)
            self._process_duals(dmu_code, self.input_data.output_categories,
                                model_solution.add_output_dual)

    def _process_duals(self, dmu_code, categories, func):
        ''' Helper function that adds duals to solution using given method func.
            Helps to avoid code duplication.

            Args:
                dmu_code (str): DMU code under consideration.
                categories (list of str): list of either input or output
                    categories.
                func (function): a function that accepts DMU code, category and
                    dual variable value and adds it to solution.
        '''
        for category in categories:
            constraint_name = self._constraints[category]
            dual = self._concrete_model.process_dual_value(
                self.lp_model.constraints[constraint_name].pi)
            func(dmu_code, category, dual)


class InputOrientedModel(object):

    ''' This class implements methods that are the same for all input-oriented
        models.
    '''
    def get_orientation(self):
        ''' Returns orientation of the model as a string.

            Returns:
                str: a string 'input'.
        '''
        return 'input'

    def process_obj_var(self, obj_value):
        ''' Post-process objective function value if necessary.

            Args:
                obj_value (double): value of the objective function.

            Returns:
                double: value of obj_variable, since efficiency score is in
                    the interval [0, 1] for input-oriented model.
        '''
        if obj_value > 1 and obj_value <= 1 + ZERO_TOLERANCE:
            return 1.0
        return obj_value

    def process_dual_value(self, dual_value):
        ''' Post-process given value of a dual variable if necessary.

            Args:
                dual_value (double): value of the dual variable.

            Returns:
                double: dual_value since in input-oriented model we do nothing
                    with duals.
        '''
        return dual_value

class OutputOrientedModel(object):
    ''' This class implements methods that are the same for all output-oriented
        models.
    '''
    def get_orientation(self):
        ''' Returns orientation of the model as a string.

            Returns:
                str: a string 'output'.
        '''
        return 'output'

    def process_obj_var(self, obj_value):
        ''' Post-process objective function value if necessary.

            Args:
                obj_value (double): value of the objective function.

            Returns:
                double: 1/obj_variable, since in the case of output-oriented
                    models the value of objective function corresponds to
                    inverse of efficiency spyDEA.core.
        '''
        if obj_value == 0:
            return float('inf')
        return 1.0/obj_value

    def process_dual_value(self, dual_value):
        ''' Post-process given value of a dual variable if necessary.

            Args:
                dual_value (double): value of the dual variable.

            Returns:
                double: -dual_value if it is non-zero since in output-oriented
                    models.
        '''
        if dual_value:
            return -dual_value
        return dual_value

class InputData:

    ''' This class stores input data and provides some useful methods to
        add input data and to access it.

        Attributes:
            DMU_codes (set of str): set of internal codes of DMUs.
                All codes are assigned automatically when DMUs are added.
            DMU_code_to_user_name (dict of str to str): dictionary that maps
                internal DMU codes to DMU names provided by the user.
            _DMU_user_name_to_code (dict of str to str): dictionary that
                maps DMU names to internal DMU codes. It is useful for
                adding new DMUs and for unit tests.
            DMU_codes_in_added_order (list of str): list of DMU codes
                in the order they were added.
            categories (set of str): set of all categories (input and
                output).
            coefficients (dict of tuple of str, str to double}): dictionary
                that maps internal DMU code and category to the
                corresponding coefficient, e.g. {(DMU, category) : value}.
            output_categories (set of str): set of output categories.
            input_categories (set of str): set of input categories.
            _count (int): internal variable used to generate DMU codes.
    '''
    def __init__(self):
        self.DMU_codes = set()
        self.DMU_code_to_user_name = dict()
        self._DMU_user_name_to_code = dict()  # intended to use
        # only in this class for adding new DMUs
        self.DMU_codes_in_added_order = []
        self.categories = set()  # all categories
        self.coefficients = dict()
        self.output_categories = set()
        self.input_categories = set()
        self._count = 0

    def add_coefficient(self, dmu_user_name, category_name, value):
        ''' Adds coefficient value corresponding to DMU and
            category to the internal data structure.
            Updates other containers that
            store DMUs and categories. If given pair of DMU and category already
            exist, KeyError is raised.

            Args:
                dmu_user_name (str): DMU name.
                category_name (str): category (input or output).
                value (double): coefficient value.

            Raises:
                KeyError: if a pair of dmu_user_name and category_name
                    has already been added before
        '''
        # if dmu_user_name is alread in the self._DMU_user_name_to_code
        # corresponding dmu_code is returned.
        # if dmu_user_name is not in self._DMU_user_name_to_code
        # new code will be generated and added to
        # self._DMU_user_name_to_code
        # self._generate_next_DMU_code() is executed every time,
        # so codes are not consecutive.
        dmu_code = self._DMU_user_name_to_code.setdefault(
            dmu_user_name, self._generate_next_DMU_code())

        key = (dmu_code, category_name)
        if key in self.coefficients:
            raise KeyError('Pair ({dmu}, {category}) is already recorded'.
                           format(dmu=dmu_code, category=category_name))

        self.coefficients[key] = value

        if dmu_code not in self.DMU_codes:
            self.DMU_codes_in_added_order.append(dmu_code)

        self.DMU_codes.add(dmu_code)
        self.DMU_code_to_user_name.setdefault(dmu_code, dmu_user_name)
        self.categories.add(category_name)

    def _generate_next_DMU_code(self):
        ''' Generates a code for new DMU in the following format: {dmu_number}.

            Returns:
                str: generated DMU code.
        '''
        self._count += 1
        return 'dmu_{index}'.format(index=self._count)

    def get_dmu_user_name(self, dmu_code):
        ''' Returns DMU user name given DMU code.

            Args:
                dmu_code (str): DMU code.

            Returns:
                str: DMU name.

            Raises:
                KeyError: if dmu_code does not exist.
        '''
        return self.DMU_code_to_user_name[dmu_code]

    def add_input_category(self, category_name):
        ''' Adds given category to the set of input categories.

            Args:
                category_name (str): name of the input category.

            Raises:
                KeyError: if category_name is not present among existing
                    categories.
        '''
        self._check_if_category_exists(category_name)
        if category_name in self.output_categories:
            raise ValueError('Category: {category} was previously added'
                             ' to output categories'.format(
                             category=category_name))
        self.input_categories.add(category_name)

    def add_output_category(self, category_name):
        ''' Adds given category to the set of output categories.

            Args:
                category_name (str): name of the output category.

            Raises:
                KeyError: if category_name is not present among existing
                    categories.
        '''
        self._check_if_category_exists(category_name)
        if category_name in self.input_categories:
            raise ValueError('Category: {category} was previously added'
                             ' to input categories'.format(
                             category=category_name))
        self.output_categories.add(category_name)

    def _check_if_category_exists(self, category_name):
        ''' Helper function that raises KeyError if a given category is not
            present in the list of categories.

            Args:
                category_name (str): category (input or output).

            Raises:
                KeyError: if category_name is not in the set of existing
                    categories.
        '''
        if category_name not in self.categories:
            raise KeyError('{category} is not present in categories list'.
                           format(category=category_name))

    def print_coefficients(self):
        ''' Prints all coefficients on the screen.
        '''
        for item in self.coefficients.items():
            print('DMU: {0}, category: {1}, value: {2}'.
                  format(item[0][0], item[0][1], item[1]))
        for dmu_code in self.DMU_codes:
            print('DMU code:{code}'.format(code=dmu_code))
        for item in self.DMU_code_to_user_name.items():
            print('DMU code: {code}, name: {name}'.
                  format(code=item[0], name=item[1]))
        for category in self.categories:
            print('category: {category}'.format(category=category))


#  This module contains concrete implementations of input- and
#  output-oriented envelopment models as well as the model with
#  non-discretionary variables.


class EnvelopmentModelInputOriented(InputOrientedModel):
    ''' This class defines methods specific to input-oriented envelopment
        model.

        Attributes:
            upper_bound_generator (function): function that
                generates an upper bound on efficiency scores for
                envelopment model, see :mod:`refactored.bound_generators`

        Args:
            upper_bound_generator (function): function that
                generates an upper bound on efficiency scores for
                envelopment
                model, see :mod:`refactored.bound_generators`.
    '''
    def __init__(self, upper_bound_generator):
        self.upper_bound_generator = upper_bound_generator

    def get_upper_bound_for_objective_variable(self):
        ''' Returns a proper upper bound on efficiency score which
            is minimized in the case of input-oriented envelopment model.

            Returns:
                double: upper bound on efficiency spyDEA.core.
        '''
        return self.upper_bound_generator()

    def get_lower_bound_for_objective_variable(self):
        ''' Returns 0 which is the lower bound on efficiency score which
            is minimized in the case of input-oriented envelopment model.

            Returns:
                double: zero.
        '''
        return 0

    def get_objective_type(self):
        ''' Returns pulp.LpMinimize - we minimize objective function in case
            of input-oriented envelopment model.

            Returns:
                pulp.LpMaximize: type of objective function.
        '''
        return pulp.LpMinimize

    def get_output_variable_coefficient(self, obj_variable, output_category):
        ''' Returns 1, since in input-oriented model we do not multiply
            current output by anything.

            Args:
                obj_variable (pulp.LpVariable): pulp variable that corresponds
                    to output category of the current DMU.
                output_category (str): output category for which current
                    constraint is being created.

            Returns:
                double: output variable coefficient.
        '''
        return 1

    def get_input_variable_coefficient(self, obj_variable, input_category):
        ''' Returns obj_variable, since in input-oriented model we multiply
            current input by efficiency spyDEA.core.

            Args:
                obj_variable (pulp.LpVariable): pulp variable that corresponds
                    to input category of current DMU.
                input_category (str): input category for which current
                    constraint is being created.

            Returns:
                pulp.LpVariable: input variable coefficient.
        '''
        return obj_variable

    def update_output_category_coefficient(self, current_output, constraint,
                                           obj_var, output_category):
        ''' Updates coefficient of a given output category with a new
            value.

            Args:
                current_output (double): new value for the coefficient.
                constraint (pulp.LpConstraint): constraint whose coefficient
                    should be updated.
                obj_var (pulp.LpVariable): variable of the envelopment
                    model that is optimised in the objective function.
                output_category (str): output category name.
        '''
        constraint.changeRHS(current_output)

    def update_input_category_coefficient(self, current_input, constraint,
                                          obj_var, input_category):
        ''' Updates coefficient of a given input category with a new
            value.

            Args:
                current_output (double): new value for the coefficient.
                constraint (pulp.LpConstraint): constraint whose coefficient
                    should be updated.
                obj_var (pulp.LpVariable): variable of the envelopment
                    model that is optimised in the objective function.
                output_category (str): input category name.
        '''
        constraint[obj_var] = current_input

class EnvelopmentModelOutputOriented(OutputOrientedModel):

    ''' This class defines methods specific
        to output-oriented envelopment model.

        Attributes:
            lower_bound_generator (function): function that
                generates a lower bound on inverse of efficiency scores
                for envelopment
                model, see :mod:`refactored.bound_generators`

        Args:
            lower_bound_generator (function): function that
                generates a lower bound on inverse of efficiency scores for
                envelopment
                model, see :mod:`refactored.bound_generators`
    '''
    def __init__(self, lower_bound_generator):
        self.lower_bound_generator = lower_bound_generator

    def get_upper_bound_for_objective_variable(self):
        ''' Returns None, since variables of output-oriented envelopment
            model are not bounded.

            Returns:
                double: None.
        '''
        return None

    def get_lower_bound_for_objective_variable(self):
        ''' Returns a proper lower bound on the variables corresponding
            to output-oriented envelopment model.

            Returns:
                double: lower bound on variables.
        '''
        return self.lower_bound_generator()

    def get_objective_type(self):
        return pulp.LpMaximize

    def get_output_variable_coefficient(self, obj_variable, output_category):
        ''' Returns obj_variable, since in output-oriented model we multiply
            current output by inverse efficiency spyDEA.core.

            Args:
                obj_variable (pulp.LpVariable): pulp variable that corresponds
                    to output category of current DMU.
                output_category (str): output category for which current
                    constraint is being created.

            Returns:
                pulp.LpVariable: output variable coefficient.
        '''
        return obj_variable

    def get_input_variable_coefficient(self, obj_variable, input_category):
        return 1

    def update_output_category_coefficient(self, current_output, constraint,
                                           obj_var, output_category):
        ''' Updates coefficient of a given output category with a new
            value.

            Args:
                current_output (double): new value for the coefficient.
                constraint (pulp.LpConstraint): constraint whose coefficient
                    should be updated.
                obj_var (pulp.LpVariable): variable of the envelopment
                    model that is optimised in the objective function.
                output_category (str): output category name.
        '''
        constraint[obj_var] = -current_output

    def update_input_category_coefficient(self, current_input, constraint,
                                          obj_var, input_category):
        ''' Updates coefficient of a given input category with a new
            value.

            Args:
                current_output (double): new value for the coefficient.
                constraint (pulp.LpConstraint): constraint whose coefficient
                    should be updated.
                obj_var (pulp.LpVariable): variable of the envelopment
                    model that is optimised in the objective function.
                output_category (str): input category name.
        '''
        constraint.changeRHS(-current_input)

class EnvelopmentModelInputOrientedWithNonDiscVars(
        EnvelopmentModelInputOriented):
    ''' This class redefines some methods of EnvelopmentModelInputOriented
        in order to take into account non-discretionary variables.

        Note:
            This class does not have a reference
            to InputData object. Hence, it cannot check if supplied
            non-discretionary categories are valid input categories.

        Attributes:
            non_disc_inputs (list of str): list of non-discretionary input
                categories.

        Args:
            non_disc_inputs (list of str): list of non-discretionary input
                categories.
            upper_bound_generator (function): function that
                generates an upper bound on efficiency scores for envelopment
                model, see :mod:`refactored.bound_generators`
    '''
    def __init__(self, non_disc_inputs, upper_bound_generator):
        super(EnvelopmentModelInputOrientedWithNonDiscVars, self).__init__(
            upper_bound_generator)
        assert(non_disc_inputs)
        self.non_disc_inputs = non_disc_inputs

    def get_input_variable_coefficient(self, obj_variable, input_category):
        ''' Returns proper coefficient depending on the fact if variable
            is discretionary or not.

            Args:
                obj_variable (pulp.LpVariable): pulp variable that corresponds
                    to input category of current DMU.
                input_category (str): input category for which current
                    constraint is being created.

            Returns:
                double or pulp.LpVariable: input variable coefficient.

        '''
        if input_category in self.non_disc_inputs:
            return 1
        return obj_variable

    def update_input_category_coefficient(self, current_input, constraint,
                                          obj_var, input_category):
        ''' Updates coefficient of a given input category with a new
            value.

            Args:
                current_output (double): new value for the coefficient.
                constraint (pulp.LpConstraint): constraint whose coefficient
                    should be updated.
                obj_var (pulp.LpVariable): variable of the envelopment
                    model that is optimised in the objective function.
                output_category (str): input category name.
        '''
        if input_category in self.non_disc_inputs:
            constraint.changeRHS(-current_input)
        else:
            constraint[obj_var] = current_input

class EnvelopmentModelOutputOrientedWithNonDiscVars(
        EnvelopmentModelOutputOriented):
    ''' This class redefines some methods of EnvelopmentModelOutputOriented
        in order to take into account non-discretionary variables.

        Note:
            This class does not have a reference
            to InputData object. Hence, it cannot check if supplied
            non-discretionary categories are valid output categories.

        Attributes:
            non_disc_outputs (list of str): list of non-discretionary output
                categories.

        Args:
            non_disc_outputs (list of str): list of non-discretionary output
                categories.
            lower_bound_generator (function): objects that
                generates a lower bound on inverse of efficiency scores for
                envelopment model, see :mod:`refactored.bound_generators`.
    '''
    def __init__(self, non_disc_outputs, lower_bound_generator):
        assert(non_disc_outputs)
        super(EnvelopmentModelOutputOrientedWithNonDiscVars, self).__init__(
            lower_bound_generator)
        self.non_disc_outputs = non_disc_outputs

    def get_output_variable_coefficient(self, obj_variable, output_category):
        ''' Returns a proper coefficient depending on the fact if the variable
            is discretionary or not.

            Args:
                obj_variable (pulp.LpVariable): pulp variable that corresponds
                    to output category of current DMU.
                input_category (str): output category for which current
                    constraint is being created.
            Returns:
                double or pulp.LpVariable: output variable coefficient.
        '''
        if output_category in self.non_disc_outputs:
            return 1
        return obj_variable

    def update_output_category_coefficient(self, current_output, constraint,
                                           obj_var, output_category):
        ''' Updates coefficient of a given output category with a new
            value.

            Args:
                current_output (double): new value for the coefficient.
                constraint (pulp.LpConstraint): constraint whose coefficient
                    should be updated.
                obj_var (pulp.LpVariable): variable of the envelopment
                    model that is optimised in the objective function.
                output_category (str): output category name.
        '''
        if output_category in self.non_disc_outputs:
            constraint.changeRHS(current_output)
        else:
            constraint[obj_var] = -current_output

In [4]:
mdl = EnvelopmentModelInputOriented(upper_bound_generator=generate_upper_bound_for_efficiency_score)
model = EnvelopmentModelBase(mdl)


TypeError: EnvelopmentModelBase.__init__() missing 2 required positional arguments: 'concrete_model' and 'constraint_creator'